In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tools.utils import *
from yolo import YOLO, detect_video
from PIL import Image
%matplotlib inline

# dict_2Dpoints # KEY = ID, VALUE = (X, Y) , all 2D points


path = '/home/kk/Downloads/2011_09_26_drive_0005_sync/2011_09_26/2011_09_26_drive_0005_sync/'
image_type = 'color' # 'grayscale' or 'color' image
mode = '00' if image_type == 'grayscale' else '02'
img = path + 'image_'+ mode +'/data/0000000100.png'

def load_from_bin(bin_path):
    obj = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    # ignore reflectivity info
    return obj[:,:3]

def load_from_bin_with_reflect(bin_path):
    obj = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    # Do NOT ignore reflectivity info
    return obj[:,:4]

def add_velo_points_array_with_id(arr_points):
    list_v = list(arr_points)
    i = 0
    list_v_id=[]
    for v in list_v:
        v=np.append(v, i)
        list_v_id.append(v)
        i=i+1
    arr_points_id = np.array(list_v_id).astype(np.float32)
    return arr_points_id

def print_projection_plt(points, color, image):
    """ project converted velodyne points into camera image """
    
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    for i in range(points.shape[1]):
        cv2.circle(hsv_image, (np.int32(points[0][i]),np.int32(points[1][i])),2, (int(color[i]),255,255),-1)

    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)

In [ ]:
# bin file -> numpy array
# velo_points = load_from_bin(path + 'velodyne_points/data/0000000050.bin')
# velo_points_with_ref = load_from_bin_with_reflect(path + 'velodyne_points/data/0000000050.bin')   

# # Assign id for every 3d points
# # velo_points_with_ref.shape    (122516, 4) x, y, z, r
# # velo_points_with_ref_id.shape (122516, 5) x, y, z, r, id

# velo_points_with_ref_id = add_velo_points_array_with_id(velo_points_with_ref)
#assert  velo_points_with_ref_id.shape == (122516, 5)

In [ ]:
# for name in glob.glob(path + 'image_02/' + "data/" + "*" ):
#     print(name)

Cloud points to 2d points

In [ ]:
path = '/home/KITTI_Tutorial/data/2011_09_26/2011_09_26_drive_0005_sync/'
path =  '/home/kk/Downloads/2011_09_26_drive_0005_sync/2011_09_26/2011_09_26_drive_0005_sync/'
v2c_filepath = '/home/KITTI_Tutorial/data/2011_09_26/calib_velo_to_cam.txt'
c2c_filepath = '/home/KITTI_Tutorial/data/2011_09_26/calib_cam_to_cam.txt'
v2c_filepath = '/home/kk/Downloads/2011_09_26_calib/2011_09_26/calib_velo_to_cam.txt'
c2c_filepath = '/home/kk/Downloads/2011_09_26_calib/2011_09_26/calib_cam_to_cam.txt'

# bin file -> numpy array
velo_points = load_from_bin(path + 'velodyne_points/data/0000000100.bin')
velo_points_with_ref = load_from_bin_with_reflect(path + 'velodyne_points/data/0000000100.bin') 

In [ ]:
velo_points_with_ref_id = add_velo_points_array_with_id(velo_points_with_ref)

arr_2Dpoints_xyz_, c_ , arr_2Dpoints_xyzri_= velo3d_2_camera2d_points(velo_points_with_ref_id, v_fov=(-24.9, 2.0), h_fov=(-45,45), \
                               vc_path=v2c_filepath, cc_path=c2c_filepath, mode=mode)

dict_3Dpoints = getDict3DPoints(velo_points_with_ref_id)

# print(len(velo_points_with_ref_id))
# print(len(arr_2Dpoints_xyzri_))
# print(len(arr_2Dpoints_xyz_))

# print((arr_2Dpoints_xyzri_))
# print((arr_2Dpoints_xyz_))

# dict_2Dpoints # KEY = ID, VALUE = (X, Y) , all 2D points
def get_dict(arr_2Dpoints_xyzri_):
    dict_r_FOV = {}
    dict_dist_FOV = {}
    dict_2Dpoints_FOV = {} 
    id_list = []
    id_list = arr_2Dpoints_xyzri_[5]
    for idx, iid in enumerate(id_list):
        dict_2Dpoints_FOV[iid] = list()
        dict_dist_FOV[iid] = list()
        dict_r_FOV[iid] = list()
        dict_2Dpoints_FOV[iid].append([arr_2Dpoints_xyzri_[0, idx], arr_2Dpoints_xyzri_[1, idx]])

        dict_dist_FOV[iid].append([dict_3Dpoints[iid][4]])
        #print([dict_3Dpoints[iid][3]])
        dict_r_FOV[iid].append([dict_3Dpoints[iid][3]])
    
    return dict_2Dpoints_FOV, dict_dist_FOV, dict_r_FOV
    
dict_2Dpoints_FOV, dict_dist_FOV, dict_r_FOV = get_dict(arr_2Dpoints_xyzri_)    
    

    
# dict_r_FOV = {}
# dict_dist_FOV = {}
# dict_2Dpoints_FOV = {} 
# id_list = []
# id_list = arr_2Dpoints_xyzri_[5]
# for idx, iid in enumerate(id_list):
#     dict_2Dpoints_FOV[iid] = list()
#     dict_dist_FOV[iid] = list()
#     dict_r_FOV[iid] = list()
#     dict_2Dpoints_FOV[iid].append([arr_2Dpoints_xyzri_[0, idx], arr_2Dpoints_xyzri_[1, idx]])
    
#     dict_dist_FOV[iid].append([dict_3Dpoints[iid][4]])
#     #print([dict_3Dpoints[iid][3]])
#     dict_r_FOV[iid].append([dict_3Dpoints[iid][3]])

In [ ]:
def get_2DpointsId_in_box(box, dict_2Dpoints_FOV):
    for inx, iid in enumerate(dict_2Dpoints_FOV):
            p = dict_2Dpoints_FOV[iid][0] # [[x, y]], p = [x, y]
            #print(p)
            # print(type(p))
            if point_in_bbox_or_not(p, box):
                    box._2DpointsId.append(iid)
    return box

# # person 0.99 (661, 179) (760, 315)
# # 661, 179, 760-661, 315-179
# # 662, 174, 104, 168

# motorbike 0.31 (310, 187) (328, 212)
# bicycle 0.95 (663, 241) (765, 338)


(left, top)     = (663, 241)
(right, bottom) = (765, 338)

box = BoundingBox('person', 0.99, left, top, right-left, bottom-top)
box = get_2DpointsId_in_box(box, dict_2Dpoints_FOV)
print(box.predicted_class)
# print(box.w)
print(len(dict_2Dpoints_FOV))
# #print((dict_2Dpoints_FOV[0]))
print(len(box._2DpointsId))

In [ ]:
# # dict_2Dpoints_FOV_Box  key:id , value:(u, v) 
# cnt = 0
# dict_2Dpoints_FOV_Box = {}
# for iid in box._2DpointsId:
#     #print(iid)
#     cnt = cnt + 1
#     dict_2Dpoints_FOV_Box[int(iid)] = dict_2Dpoints_FOV[iid]
# #print((dict_2Dpoints_FOV_Box))

# #dict_dist_FOV_Box
# dict_dist_FOV_Box = {}
# for iid in box._2DpointsId:
#     dict_dist_FOV_Box[iid] = dict_dist_FOV[iid]
# print(len(dict_dist_FOV_Box))

# #dict_r_FOV_Box
# dict_r_FOV_Box = {}
# for iid in box._2DpointsId:
#     dict_r_FOV_Box[iid] = dict_r_FOV[iid]
# print(len(dict_r_FOV_Box))

# list_2Dpoints_Box = []
# list_dist_Box     = []
# list_r_Box        = []
# for iid in box._2DpointsId:
#     list_2Dpoints_Box.append(dict_2Dpoints_FOV_Box[iid])
#     list_dist_Box.append(dict_dist_FOV_Box[iid])
#     list_r_Box.append(dict_r_FOV_Box[iid])
    
    
# arr_2Dpoints_Box = np.array(list_2Dpoints_Box)
# arr_dist_Box     = np.array(list_dist_Box)
# arr_r_Box        = np.array(list_r_Box) 
# len(arr_2Dpoints_Box)

# n_points = len(arr_2Dpoints_Box)
# print(n_points)

In [ ]:
def get_point_in_box(box, dict_2Dpoints_FOV, dict_dist_FOV, dict_r_FOV):
    # dict_2Dpoints_FOV_Box  key:id , value:(u, v) 
    
    cnt = 0
    dict_2Dpoints_FOV_Box = {}
    for iid in box._2DpointsId:
        #print(iid)
        cnt = cnt + 1
        dict_2Dpoints_FOV_Box[int(iid)] = dict_2Dpoints_FOV[iid]
    #print((dict_2Dpoints_FOV_Box))

    #dict_dist_FOV_Box
    dict_dist_FOV_Box = {}
    for iid in box._2DpointsId:
        dict_dist_FOV_Box[iid] = dict_dist_FOV[iid]
    print(len(dict_dist_FOV_Box))

    #dict_r_FOV_Box
    dict_r_FOV_Box = {}
    for iid in box._2DpointsId:
        dict_r_FOV_Box[iid] = dict_r_FOV[iid]
    print(len(dict_r_FOV_Box))

    list_2Dpoints_Box = []
    list_dist_Box     = []
    list_r_Box        = []
    for iid in box._2DpointsId:
        list_2Dpoints_Box.append(dict_2Dpoints_FOV_Box[iid])
        list_dist_Box.append(dict_dist_FOV_Box[iid])
        list_r_Box.append(dict_r_FOV_Box[iid])

    arr_2Dpoints_Box = np.array(list_2Dpoints_Box)
    arr_dist_Box     = np.array(list_dist_Box)
    arr_r_Box        = np.array(list_r_Box) 
    len(arr_2Dpoints_Box)

    n_points = len(arr_2Dpoints_Box)
    print(n_points)
    
    return arr_2Dpoints_Box, dict_2Dpoints_FOV_Box, dict_dist_FOV_Box
    
###

arr_2Dpoints_Box, dict_2Dpoints_FOV_Box, dict_dist_FOV_Box = get_point_in_box(box, dict_2Dpoints_FOV, dict_dist_FOV, dict_r_FOV)

# print(len(arr_2Dpoints_Box))
# print((arr_2Dpoints_Box))

In [ ]:
from tools.utils import *
# divid_bins = 2 
# n_points_threshold = 1000
# print(len(dict_dist_FOV_Box))
# print(len(dict_2Dpoints_FOV_Box))
# print(len(dict_3Dpoints))

# Key operation 
#, dist, uvd_list
arr_top  = get_uv_dist_from_box(box, dict_3Dpoints, dict_2Dpoints_FOV_Box, dict_dist_FOV_Box, divid_bins=2, n_points_threshold=500)
print(arr_top)
#uc , vc, xmid, ymid = estimate_shape_center(box, dist, dict_2Dpoints_FOV_Box, dict_dist_FOV_Box)

# Image about distance

In [ ]:
## Test detector
# from tools.utils import *

# def detector(yolo, img, dict_2Dpoints_FOV, arr_2Dpoints_xyzri_, dict_3Dpoints):
#     image = Image.open(img)
#     print(image)
    
#     r_image = yolo.detect_image(image, dict_2Dpoints_FOV, arr_2Dpoints_xyzri_, dict_3Dpoints)
#     r_image.show()

#     yolo.close_session()


# detector(YOLO(), img, dict_2Dpoints_FOV, arr_2Dpoints_xyzri_, dict_3Dpoints)

# Video about distance

In [ ]:
video_path = '/home/kk/Work/MegaDepth_Tensorflow/video/video_from_images_KITTI.avi'
output_path="/home/kk/Downloads/tt.avi" # test3 is image with 3d points on it first and then detection.
detect_video(YOLO(), video_path, output_path, dict_2Dpoints_FOV, arr_2Dpoints_xyzri_, dict_3Dpoints)